# package_DBR

In [19]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from matplotlib import colors as mcolors
import os
import package_DBR
import PACKAGE_LAB

from importlib import reload
package_DBR = reload(package_DBR)
PACKAGE_LAB = reload(PACKAGE_LAB)

from package_DBR import *
from PACKAGE_LAB import *

In [20]:
help(LL_RT)

Help on function LL_RT in module PACKAGE_LAB:

LL_RT(MV, Kp, TLead, TLag, Ts, PV, PVInit=0, method='EBD')
    The function "LL_RT" needs to be included in a "for or while loop".
    
    :MV: input vector
    :Kp: process gain
    :T1: lead time constant [s]
    :TLead: lead time constant [s]
    :TLag: lag time constant [s] 
    :Ts: sampling period [s]
    :PV: output vector
    :PVInit: (optional: default value is 0)
    :method: discretisation method (optional: default value is 'EBD')
        EBD: Euler Backward difference
        EFD: Euler Forward difference
        TRAP: Trapezoïdal method
    
    The function "LL_RT" appends a value to the output vector "PV".
    The appended value is obtained from a recurrent equation that depends on the discretisation method.



In [21]:
import ipywidgets as widgets
from IPython.display import display

MVPath = {0: 0, 5:3, TSim: 3}
# Function to generate graphs based on parameters
def generate_graphs(Tlead_1, Tlead_2, Tlead_3, Tlag, theta):
    t = []
    MV = []
    MVDelay = []
    PV_EBD_LEAD1 = []
    PV_EBD_LEAD2 = []
    PV_EBD_LEAD3 = []

    for i in range(0, N):
        t.append(i * Ts)
        SelectPath_RT(MVPath, t, MV)
        Delay_RT(MV, theta, Ts, MVDelay)
        LL_RT(MVDelay, Kp, Tlead_1, Tlag, Ts, PV_EBD_LEAD1)
        LL_RT(MVDelay, Kp, Tlead_2, Tlag, Ts, PV_EBD_LEAD2)
        LL_RT(MVDelay, Kp, Tlead_3, Tlag, Ts, PV_EBD_LEAD3)

    plt.figure(figsize=(15, 9))

    plt.subplot(2, 1, 1)
    plt.step(t, MV, 'b-', label='MV', where='post')
    plt.step(t, MVDelay, 'b-', linewidth=0.5, label='MV plus delay', where='post')
    plt.ylabel('Value of MV')
    plt.title('Lead-Lag plus delay response')
    plt.legend(loc='best')
    plt.xlim([0, TSim])

    plt.subplot(2, 1, 2)
    plt.step(t, PV_EBD_LEAD1, 'r-', label='LL lead 1(EBD)', where='post')
    plt.step(t, PV_EBD_LEAD2, 'g-', label='LL lead 2(EBD)', where='post')
    plt.step(t, PV_EBD_LEAD3, 'b-', label='LL lead 3(EBD)', where='post')

    plt.ylabel('PV')
    plt.xlabel('Time [s]')
    plt.legend(loc='best')
    plt.xlim([0, TSim])

    plt.show()

# Create widgets for parameter adjustment
Tlead_1_widget = widgets.FloatSlider(value=15, min=-100, max=100, step=0.1, description='Tlead_1:')
Tlead_2_widget = widgets.FloatSlider(value=5, min=-100, max=100, step=0.1, description='Tlead_2:')
Tlead_3_widget = widgets.FloatSlider(value=-2, min=-100, max=100, step=0.1, description='Tlead_3:')
Tlag_widget = widgets.FloatSlider(value=10, min=-100, max=100, step=0.1, description='Tlag:')
theta_widget = widgets.FloatSlider(value=10, min=-100, max=100, step=0.1, description='Theta:')

# Create an interactive widget
widgets.interactive(generate_graphs, Tlead_1=Tlead_1_widget, Tlead_2=Tlead_2_widget, Tlead_3=Tlead_3_widget, Tlag=Tlag_widget, theta=theta_widget)


interactive(children=(FloatSlider(value=15.0, description='Tlead_1:', min=-100.0), FloatSlider(value=5.0, desc…

In [27]:
# Define simulation parameters
TSim = 100
Ts = 0.5
N = int(TSim / Ts) + 1

# Define MV & SP path
SPPath = {0: 50, 5: 60, TSim: 60} 
PVpath = {0: 50, TSim: 50}   
ManPath = {0: False, TSim: False} 
MVManPath = {0: 0, TSim: 0}
MVFFPath = {0: 0, TSim: 0}

# Initialize lists
SP = []
PV = []
Man = []
MVMan = []
MVFF = []
MV = []
MVp = []
MVi = []
MVd = []
E = []

Kc=5
TD=50
alpha=0.9

# Set maximum and minimum MV values
MVmin = 0
MVmax = 100

for i in range(0, N):
    t.append(i * Ts)
    SelectPath_RT(SPPath, t, SP)
    SelectPath_RT(PVpath, t, PV)
    SelectPath_RT(ManPath, t, Man)
    SelectPath_RT(MVManPath, t, MVMan)
    SelectPath_RT(MVFFPath, t, MVFF)
TI = TD/0.25
 # Generate time vector and simulate MV, MVp, MVi, MVd, and E
for i in range(0, N):
    PID_RT(SP[0:i+1], PV[0:i+1], Man[0:i+1], MVMan[0:i+1], MVFF[0:i+1], Kc, TI, TD, alpha, Ts, MVmin, MVmax, MV, MVp, MVi, MVd, E)


# Plot results
# Plotting
plt.figure(figsize=(10, 6))

plt.subplot(2, 3, 1)
plt.plot(t, SP)
plt.title('Setpoint (SP)')
plt.xlabel('Time')
plt.ylabel('Value')

plt.subplot(2, 3, 2)
plt.plot(t, PV)
plt.title('Process Variable (PV)')
plt.xlabel('Time')
plt.ylabel('Value')

plt.subplot(2, 3, 3)
plt.plot(t, Man)
plt.title('Manipulated Variable (Man)')
plt.xlabel('Time')
plt.ylabel('Value')

plt.subplot(2, 3, 4)
plt.plot(t, MVMan)
plt.title('MV from Manual Control (MVMan)')
plt.xlabel('Time')
plt.ylabel('Value')

plt.subplot(2, 3, 5)
plt.plot(t, MVFF)
plt.title('MV from Feedforward (MVFF)')
plt.xlabel('Time')
plt.ylabel('Value')

plt.subplot(2, 3, 6)
plt.plot(t, MV, label='MV')
plt.plot(t, MVp, label='MVp')
plt.plot(t, MVi, label='MVi')
plt.plot(t, MVd, label='MVd')
plt.plot(t, E, label='E')
plt.title('PID Components')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()

plt.tight_layout()
plt.show()


NameError: name 'TD' is not defined